# Streamlit: создание интерактивных ML-демонстраций



После разработки ML-модели часто возникает необходимость продемонстрировать её работу коллегам, заказчикам или пользователям. Традиционные подходы имеют свои ограничения:

**Jupyter notebook**
- Требует установки Python и зависимостей
- Неудобен для нетехнических пользователей
- Сложно делиться результатами

**Flask/FastAPI + HTML/CSS/JavaScript**
- Требует знания веб-разработки
- Долго создавать даже простой интерфейс
- Нужно писать frontend и backend отдельно

**Готовые платформы (Hugging Face Spaces, Gradio)**
- Ограниченная кастомизация
- Зависимость от внешних сервисов
- Не всегда подходят для специфичных задач



Streamlit это Python-фреймворк для быстрого создания интерактивных веб-приложений. Ключевые преимущества:

- Только Python, не нужны HTML, CSS, JavaScript
- Быстрая разработка
- Автоматическая реактивность, изменения параметров мгновенно обновляют результаты
- Богатый набор виджетов: слайдеры, кнопки, загрузка файлов, графики
- Встроенное кеширование
- Простой деплой на Streamlit Cloud, Hugging Face Spaces или собственном сервере




В этом блокноте мы научимся:
1. Создавать базовые Streamlit приложения
2. Работать с различными виджетами ввода
3. Визуализировать данные и результаты
4. Интегрировать ML-модели
5. Создавать полноценные ML-демонстрации

Полезные ссылки:
* [Официальная документация Streamlit](https://docs.streamlit.io/)
* [Галерея примеров](https://streamlit.io/gallery)
* [Cheat Sheet](https://docs.streamlit.io/library/cheatsheet)

Установка:

In [ ]:
!pip install -q streamlit torch torchvision pillow matplotlib numpy pandas plotly

## Запуск Streamlit в Colab

Streamlit создаёт веб-сервер, который в Colab нужно туннелировать. Мы будем использовать `pyngrok` для создания публичного URL.

In [ ]:
!pip install -q pyngrok

In [ ]:
# Добавить отдельную ячейку после установки pyngrok
from pyngrok import ngrok

# Замените на свой токен с https://dashboard.ngrok.com/get-started/your-authtoken
NGROK_AUTH_TOKEN = "ВАШ_ТОКЕН_ЗДЕСЬ"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

In [ ]:
def run_streamlit_app(script_name):
    from pyngrok import ngrok
    import subprocess
    import time
    import os
    import signal

    # Убиваем старые туннели
    ngrok.kill()

    # ВАЖНО: Убиваем старые процессы Streamlit, чтобы освободить порт
    # В Colab это делается через системную команду pkill
    subprocess.run(["pkill", "-9", "streamlit"])

    # Запускаем Streamlit в фоне
    process = subprocess.Popen(
        ["streamlit", "run", script_name, "--server.port", "8501", "--server.headless", "true"],
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE
    )

    # Даём время на запуск
    time.sleep(3)

    try:
        # Создаём туннель
        public_url = ngrok.connect(8501).public_url
        print(f"\n Приложение '{script_name}' запущено!")
        print(f" Ссылка: {public_url}")
        print(f"\n Не останавливайте эту ячейку, пока тестируете приложение.")
    except Exception as e:
        print(f"Ошибка Ngrok: {e}")
        print("Проверьте, вставили ли вы токен авторизации.")

    return process

## Пример 1: Hello World

Начнём с простейшего Streamlit приложения отображения текста и базовых элементов.

In [ ]:
%%writefile app_hello.py
import streamlit as st

# Заголовок приложения
st.title("Моё первое Streamlit приложение")

# Различные типы текста
st.header("Это заголовок")
st.subheader("Это подзаголовок")
st.text("Это обычный текст")
st.markdown("Это **Markdown** текст с *форматированием*")

# Разделитель
st.divider()

# Код
st.code("""
def hello():
    print("Hello, Streamlit!")
""", language="python")

# LaTeX формулы
st.latex(r"E = mc^2")

# Информационные блоки
st.success("Это успешное сообщение")
st.info("Это информационное сообщение")
st.warning("Это предупреждение")
st.error("Это сообщение об ошибке")

Overwriting app_hello.py


In [ ]:
# Запускаем приложение
process = run_streamlit_app("app_hello.py")


 Приложение 'app_hello.py' запущено!
 Ссылка: https://synchronistic-cyphellate-eleonora.ngrok-free.dev

 Не останавливайте эту ячейку, пока тестируете приложение.


## Пример 2: Интерактивные виджеты

Одна из фишек Streamlit это интерактивные виджеты. При изменении любого виджета, весь скрипт перезапускается с новыми значениями.

In [ ]:
%%writefile app_widgets.py
import streamlit as st
import numpy as np
import matplotlib.pyplot as plt

st.title("Интерактивные виджеты Streamlit")

# Sidebar для виджетов
st.sidebar.header("Параметры")

# Слайдер
frequency = st.sidebar.slider(
    "Частота сигнала",
    min_value=1,
    max_value=10,
    value=5,
    step=1
)

# Number input
amplitude = st.sidebar.number_input(
    "Амплитуда",
    min_value=0.1,
    max_value=5.0,
    value=1.0,
    step=0.1
)

# Selectbox
wave_type = st.sidebar.selectbox(
    "Тип волны",
    ["Синус", "Косинус", "Квадратная"]
)

# Radio buttons
noise = st.sidebar.radio(
    "Добавить шум?",
    ["Нет", "Слабый", "Сильный"]
)

# Checkbox
show_grid = st.sidebar.checkbox("Показать сетку", value=True)

# Генерируем сигнал
x = np.linspace(0, 4*np.pi, 1000)

if wave_type == "Синус":
    y = amplitude * np.sin(frequency * x)
elif wave_type == "Косинус":
    y = amplitude * np.cos(frequency * x)
else:  # Квадратная
    y = amplitude * np.sign(np.sin(frequency * x))

# Добавляем шум
if noise == "Слабый":
    y += np.random.normal(0, 0.1, len(y))
elif noise == "Сильный":
    y += np.random.normal(0, 0.3, len(y))

# Визуализация
st.subheader("Результат")

fig, ax = plt.subplots(figsize=(10, 4))
ax.plot(x, y, linewidth=2)
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_title(f"{wave_type} волна (частота={frequency}, амплитуда={amplitude})")
if show_grid:
    ax.grid(True, alpha=0.3)

st.pyplot(fig)

# Показываем статистику
col1, col2, col3 = st.columns(3)
with col1:
    st.metric("Среднее", f"{np.mean(y):.3f}")
with col2:
    st.metric("Минимум", f"{np.min(y):.3f}")
with col3:
    st.metric("Максимум", f"{np.max(y):.3f}")

Overwriting app_widgets.py


In [ ]:
process = run_streamlit_app("app_widgets.py")


 Приложение 'app_widgets.py' запущено!
 Ссылка: https://synchronistic-cyphellate-eleonora.ngrok-free.dev

 Не останавливайте эту ячейку, пока тестируете приложение.


### Доступные виджеты



**Ввод чисел:**
- `st.slider()` — слайдер для выбора значения
- `st.number_input()` — поле для ввода числа
- `st.select_slider()` — слайдер с предопределёнными значениями

**Выбор:**
- `st.selectbox()` — выпадающий список
- `st.multiselect()` — множественный выбор
- `st.radio()` — радио-кнопки
- `st.checkbox()` — чекбокс

**Текст:**
- `st.text_input()` — однострочный текст
- `st.text_area()` — многострочный текст

**Файлы:**
- `st.file_uploader()` — загрузка файлов
- `st.camera_input()` — фото с камеры

**Другое:**
- `st.button()` — кнопка
- `st.download_button()` — скачивание файла
- `st.date_input()` / `st.time_input()` — выбор даты/времени
- `st.color_picker()` — выбор цвета

## Пример 3: Визуализация данных

Streamlit поддерживает различные библиотеки для визуализации: Matplotlib, Plotly, Altair, а также имеет встроенные функции для простых графиков.

In [ ]:
%%writefile app_visualization.py
import streamlit as st
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

st.title("Визуализация данных в Streamlit")

# Генерируем данные
np.random.seed(42)
n_points = st.slider("Количество точек", 100, 1000, 500)

df = pd.DataFrame({
    'x': np.random.randn(n_points),
    'y': np.random.randn(n_points),
    'category': np.random.choice(['A', 'B', 'C'], n_points),
    'value': np.random.uniform(10, 100, n_points)
})

# Вкладки для разных типов визуализации
tab1, tab2, tab3, tab4 = st.tabs(["Таблица", "Встроенные графики", "Plotly", "Метрики"])

with tab1:
    st.subheader("Данные")
    st.dataframe(df.head(100))  # Интерактивная таблица

    # Можно редактировать!
    st.subheader("Редактируемая таблица")
    edited_df = st.data_editor(df.head(10))

with tab2:
    st.subheader("Встроенные графики Streamlit")

    # Line chart
    st.line_chart(df.groupby('category')['value'].mean())

    # Area chart
    st.area_chart(df.groupby('category')['value'].sum())

    # Bar chart
    st.bar_chart(df.groupby('category').size())

with tab3:
    st.subheader("Plotly графики")

    # Scatter plot
    fig_scatter = px.scatter(
        df,
        x='x',
        y='y',
        color='category',
        size='value',
        title='Scatter plot с категориями'
    )
    st.plotly_chart(fig_scatter)

    # Histogram
    fig_hist = px.histogram(
        df,
        x='value',
        color='category',
        title='Распределение значений'
    )
    st.plotly_chart(fig_hist)

    # Box plot
    fig_box = px.box(
        df,
        x='category',
        y='value',
        title='Box plot по категориям'
    )
    st.plotly_chart(fig_box)

with tab4:
    st.subheader("Метрики")

    col1, col2, col3 = st.columns(3)

    with col1:
        st.metric(
            label="Среднее значение",
            value=f"{df['value'].mean():.2f}",
            delta=f"{df['value'].std():.2f}"
        )

    with col2:
        st.metric(
            label="Категория A",
            value=len(df[df['category'] == 'A']),
            delta=f"{len(df[df['category'] == 'A']) / len(df) * 100:.1f}%"
        )

    with col3:
        st.metric(
            label="Максимум",
            value=f"{df['value'].max():.2f}"
        )

Overwriting app_visualization.py


In [ ]:
process = run_streamlit_app("app_visualization.py")


 Приложение 'app_visualization.py' запущено!
 Ссылка: https://synchronistic-cyphellate-eleonora.ngrok-free.dev

 Не останавливайте эту ячейку, пока тестируете приложение.


## Пример 4: Интеграция ML-модели

Создадим приложение для классификации рукописных цифр MNIST. Покажем как загружать модель, обрабатывать пользовательский ввод и отображать результаты.

In [ ]:
# Сначала обучим простую модель
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

class SimpleCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.fc1 = nn.Linear(1600, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.max_pool2d(x, 2)
        x = torch.relu(self.conv2(x))
        x = torch.max_pool2d(x, 2)
        x = torch.flatten(x, 1)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Быстрое обучение на CPU
print("Обучение модели...")
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

train_dataset = datasets.MNIST('./data', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)

model = SimpleCNN()
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

# Обучаем 1 эпоху для демонстрации
model.train()
for batch_idx, (data, target) in enumerate(train_loader):
    optimizer.zero_grad()
    output = model(data)
    loss = criterion(output, target)
    loss.backward()
    optimizer.step()

    if batch_idx % 100 == 0:
        print(f"Batch {batch_idx}/{len(train_loader)}")

# Сохраняем модель
torch.save(model.state_dict(), 'mnist_model.pth')
print("Модель сохранена!")

Обучение модели...
Batch 0/469
Batch 100/469
Batch 200/469
Batch 300/469
Batch 400/469
Модель сохранена!


In [ ]:
%%writefile app_mnist.py
import streamlit as st
import torch
import torch.nn as nn
from PIL import Image, ImageOps
import numpy as np
import matplotlib.pyplot as plt
from torchvision import transforms

# Определение модели (должно совпадать с обучением)
class SimpleCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.fc1 = nn.Linear(1600, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.max_pool2d(x, 2)
        x = torch.relu(self.conv2(x))
        x = torch.max_pool2d(x, 2)
        x = torch.flatten(x, 1)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Кеширование загрузки модели
@st.cache_resource
def load_model():
    """Загружает модель один раз и кеширует"""
    model = SimpleCNN()
    model.load_state_dict(torch.load('mnist_model.pth', map_location='cpu'))
    model.eval()
    return model

def preprocess_image(image):
    """Подготовка изображения для модели"""
    # Конвертируем в grayscale
    image = ImageOps.grayscale(image)

    # Resize до 28x28
    image = image.resize((28, 28))

    # Инвертируем цвета (MNIST имеет белые цифры на чёрном фоне)
    image = ImageOps.invert(image)

    # Преобразуем в тензор и нормализуем
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])

    tensor = transform(image).unsqueeze(0)  # Добавляем batch dimension
    return tensor, np.array(image)

# Интерфейс приложения
st.title("Классификация рукописных цифр MNIST")
st.write("Загрузите изображение цифры или используйте тестовые примеры")

# Загрузка модели
model = load_model()

# Sidebar с информацией
st.sidebar.header("О приложении")
st.sidebar.info(
    "Это демонстрация классификации рукописных цифр "
    "с использованием CNN модели, обученной на MNIST."
)

# Выбор источника изображения
option = st.radio(
    "Выберите способ ввода:",
    ["Загрузить файл", "Использовать тестовый пример"]
)

image = None

if option == "Загрузить файл":
    uploaded_file = st.file_uploader(
        "Загрузите изображение цифры",
        type=["png", "jpg", "jpeg"]
    )
    if uploaded_file is not None:
        image = Image.open(uploaded_file)

else:
    # Загружаем тестовый пример из MNIST
    from torchvision import datasets

    @st.cache_data
    def load_test_samples():
        test_dataset = datasets.MNIST('./data', train=False, download=True)
        return test_dataset

    test_dataset = load_test_samples()

    sample_idx = st.slider(
        "Выберите тестовый пример",
        0,
        len(test_dataset) - 1,
        0
    )

    image, true_label = test_dataset[sample_idx]
    st.info(f"Истинная метка: {true_label}")

# Если есть изображение, делаем предсказание
if image is not None:
    col1, col2 = st.columns(2)

    with col1:
        st.subheader("Исходное изображение")
        st.image(image, width=200)

    # Предобработка
    tensor, processed_image = preprocess_image(image)

    with col2:
        st.subheader("После обработки (28x28)")
        st.image(processed_image, width=200)

    # Предсказание
    with torch.no_grad():
        output = model(tensor)
        probabilities = torch.softmax(output, dim=1)[0]
        prediction = torch.argmax(probabilities).item()
        confidence = probabilities[prediction].item()

    # Результаты
    st.divider()
    st.subheader("Результат классификации")

    col1, col2 = st.columns(2)

    with col1:
        st.metric(
            "Предсказанная цифра",
            prediction,
            delta=None
        )
        st.metric(
            "Уверенность",
            f"{confidence*100:.2f}%"
        )

    with col2:
        # График вероятностей
        st.subheader("Вероятности классов")

        fig, ax = plt.subplots(figsize=(6, 4))
        bars = ax.bar(
            range(10),
            probabilities.numpy(),
            color=['green' if i == prediction else 'gray' for i in range(10)]
        )
        ax.set_xlabel("Цифра")
        ax.set_ylabel("Вероятность")
        ax.set_xticks(range(10))
        ax.set_ylim([0, 1])
        ax.grid(axis='y', alpha=0.3)

        st.pyplot(fig)

    # Детальная таблица вероятностей
    with st.expander("Показать все вероятности"):
        import pandas as pd

        prob_df = pd.DataFrame({
            'Цифра': range(10),
            'Вероятность': [f"{p*100:.2f}%" for p in probabilities.numpy()]
        })
        st.dataframe(prob_df, hide_index=True)

else:
    st.info("Загрузите изображение или выберите тестовый пример для начала")

Overwriting app_mnist.py


In [ ]:
process = run_streamlit_app("app_mnist.py")


 Приложение 'app_mnist.py' запущено!
 Ссылка: https://synchronistic-cyphellate-eleonora.ngrok-free.dev

 Не останавливайте эту ячейку, пока тестируете приложение.


## Кеширование для оптимизации

Streamlit перезапускает скрипт при каждом изменении виджета. Для тяжёлых операций (загрузка модели, загрузка данных) используется кеширование:

**`@st.cache_data`** — для данных (DataFrame, списки, словари)
- Сериализует результат
- Использует для данных, которые можно преобразовать в JSON/pickle

**`@st.cache_resource`** — для ресурсов (модели, соединения с БД)
- НЕ сериализует результат
- Использует для объектов, которые нельзя сериализовать (PyTorch модели)

Пример:
```python
@st.cache_resource
def load_model():
    model = YourModel()
    model.load_state_dict(torch.load('model.pth'))
    return model

@st.cache_data
def load_data():
    return pd.read_csv('data.csv')
```

# Деплой приложения

## Hugging Face Spaces (Рекомендуется для ML)




Это самый простой и надежный способ для ML-проектов, особенно если файлы весов модели весят больше 100 МБ.

1.  Зарегистрируйтесь на [huggingface.co](https://huggingface.co/).
2.  Создайте новый **Space** (кнопка "New Space" в профиле).
3.  Введите имя проекта, выберите SDK **Streamlit**.
4.  Выберите "Public" (чтобы приложение было доступно всем).
5.  Загрузите файлы прямо через браузер ("Files" -> "Add file" -> "Upload files"):
    * `app.py` (ваш код).
    * `requirements.txt` (список библиотек).
    * `model.pth` (ваша модель).
6.  После загрузки приложение соберется и запустится автоматически.

## Streamlit Community Cloud



Работает в связке с GitHub. Идеально для легких проектов и портфолио.

**Шаг 1. Подготовка файлов**
В вашем репозитории GitHub должны лежать:
1.  `app.py` — основной код.
2.  `requirements.txt` — список библиотек Python.
    ```text
    streamlit
    torch
    torchvision
    opencv-python-headless
    pillow
    numpy
    ```
3.  **ВАЖНО для OpenCV:** Создайте файл `packages.txt`. Без него OpenCV в облаке не запустится (ошибка `libGL.so.1`).
    ```text
    libgl1
    libglib2.0-0
    ```

**Шаг 2. Загрузка весов модели**
GitHub **блокирует** файлы больше 100 МБ. Если ваша модель тяжелая:
* **Вариант А:** Используйте **Git LFS** (Large File Storage).
* **Вариант Б:** Загрузите веса на Google Drive / Яндекс.Диск, получите прямую ссылку на скачивание и добавьте в начало `app.py` код для автоматического скачивания файла при первом запуске.

**Шаг 3. Публикация**
1.  Зайдите на [share.streamlit.io](https://share.streamlit.io/).
2.  Подключите свой GitHub аккаунт.
3.  Выберите репозиторий, ветку и главный файл (`app.py`).
4.  Нажмите "Deploy".

## Docker




Если вы хотите запустить приложение на арендованном сервере (VPS). Для работы CV-библиотек нужно установить системные зависимости.

Пример правильного `Dockerfile`:

```dockerfile
# Используем легкий образ Python
FROM python:3.10-slim

WORKDIR /app

# 1. Установка системных библиотек для OpenCV (обязательно!)
RUN apt-get update && apt-get install -y \
    libgl1-mesa-glx \
    libglib2.0-0 \
    && rm -rf /var/lib/apt/lists/*

# 2. Установка Python зависимостей
COPY requirements.txt .
RUN pip install --no-cache-dir -r requirements.txt

# 3. Копирование кода приложения
COPY . .

# Открываем порт
EXPOSE 8501

# Команда запуска
CMD ["streamlit", "run", "app.py", "--server.port=8501", "--server.address=0.0.0.0"]

# Заключение

В этом блокноте мы изучили Streamlit инструмент для создания интерактивных ML-приложений на Python.

**Когда использовать Streamlit:**
- Быстро превратить ML-модель в демонстрацию
- Создавать демонстрации интерактивного анализа данных
- Делиться результатами с коллегами

**Когда НЕ использовать:**
- Нужен сложный UI с нестандартным дизайном
- Высоконагруженное production приложение
- Требуется детальный контроль над frontend
- Нужна интеграция со сложными системами аутентификации

Полезные ресурсы:

- [Официальная документация](https://docs.streamlit.io/)
- [Галерея примеров](https://streamlit.io/gallery)
- [Форум сообщества](https://discuss.streamlit.io/)
- [Awesome Streamlit](https://github.com/MarcSkovMadsen/awesome-streamlit)
